In [1]:
import sys
sys.path.append('/home/jovyan/ChestXray-14')

In [2]:
import tensorflow as tf
from modules.models import Model
from modules.utils import get_dataset
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler

2023-02-17 07:45:03.789592: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
INPUT_PATH = "/home/jovyan/ChestXray-14/dataset/ChestXray NIH"
LABELS = ['No Finding', 'Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening', 'Cardiomegaly', 'Nodule', 'Mass', 'Hernia']

In [4]:
class Dataset:
    INPUT_PATH = INPUT_PATH
    
    def get_train(self):
        filenames = tf.io.gfile.glob(f'{self.INPUT_PATH}/data/under_sampling/train/*.tfrec')
        dataset = get_dataset(filenames)
        return dataset

    def get_test(self):
        filenames = tf.io.gfile.glob(f'{self.INPUT_PATH}/data/under_sampling/test/*.tfrec')
        dataset = get_dataset(filenames)
        return dataset

## Train Model

In [5]:
import os
CURRENT_PATH = os.path.abspath("")
CURRENT_PATH

'/home/jovyan/ChestXray-14/Sampling'

In [6]:
# Learning rate
def lr_schedule(epoch, learning_rate):
    tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
    return learning_rate

In [7]:
def get_callbacks(NAME, weight_option, fold_num=None):
    model_checkpoint_callback = ModelCheckpoint(f'results/models/CrossEntropy/{NAME}_{weight_option}.h5', monitor='val_loss', mode='min', save_best_only=True)
    early_stop_callback = EarlyStopping(monitor='val_loss', mode="min", patience=20, verbose=1)
    reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', mode="min", factor=0.5, patience=3, verbose=1)
    lr_logging_callback = LearningRateScheduler(lr_schedule)
    
    return model_checkpoint_callback, early_stop_callback, reduce_lr_callback, lr_logging_callback
    

In [8]:
# Constant variables
NAME = "EfficientNetB0"
EPOCHS = 100
NUM_FOLDS = 5

In [9]:
weight_option = None # use `imagenet` or `None` only

In [10]:
# Callbacks
model_checkpoint_callback, early_stop_callback, reduce_lr_callback, lr_logging_callback = get_callbacks(NAME, weight_option)

# Path for CSV
path = os.path.join(CURRENT_PATH, "results", "history", "training_with_CrossEntroy", f"{NAME}_{weight_option}")
os.makedirs(path, exist_ok=True)

# CSV Logger
csv_logger = CSVLogger(os.path.join(path, f"CrossEntropy.csv"))

# Dataset
train_dataset, test_dataset = Dataset().get_train(), Dataset().get_test()

# Modeling
transfer_model = tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=False, 
    weights=weight_option,
    input_shape=(224, 224, 3),
    pooling=None
)

model = Model(
    transfer_model,
)
model = model.get_model(flatten=True)
model._name = "CrossEntropyOnSamplingDataset"
model.summary()

2023-02-17 07:45:06.214219: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-17 07:45:07.975649: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31741 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:b1:00.0, compute capability: 8.0


Model: "CrossEntropyOnSamplingDataset"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 dense (Dense)               (None, 128)               8028288   
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 1

In [11]:
# Visualize
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=test_dataset,
    verbose=1, # Show Progress Bar while Traning
    callbacks=[model_checkpoint_callback, csv_logger, early_stop_callback, reduce_lr_callback, lr_logging_callback]
)

Epoch 1/100


2023-02-17 07:45:20.619110: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-02-17 07:45:21.752355: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-17 07:45:21.753274: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-17 07:45:21.753292: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-02-17 07:45:21.754073: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-17 07:45:21.754221: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-02-17 07:45:25.873367: I tensorflow/stream_executor/cuda/c

3060/3060 [==============================] - 670s 213ms/step - loss: 0.2896 - auc: 0.5297 - val_loss: 0.2823 - val_auc: 0.5844 - lr: 0.0010
Epoch 2/100
3060/3060 [==============================] - 641s 209ms/step - loss: 0.2801 - auc: 0.5828 - val_loss: 0.2796 - val_auc: 0.5914 - lr: 0.0010
Epoch 3/100
3060/3060 [==============================] - 618s 202ms/step - loss: 0.2767 - auc: 0.6083 - val_loss: 0.2743 - val_auc: 0.6265 - lr: 0.0010
Epoch 4/100
3060/3060 [==============================] - 641s 210ms/step - loss: 0.2732 - auc: 0.6312 - val_loss: 0.2729 - val_auc: 0.6400 - lr: 0.0010
Epoch 5/100
3060/3060 [==============================] - 644s 210ms/step - loss: 0.2713 - auc: 0.6426 - val_loss: 0.2694 - val_auc: 0.6517 - lr: 0.0010
Epoch 6/100
3060/3060 [==============================] - 637s 208ms/step - loss: 0.2686 - auc: 0.6598 - val_loss: 0.2697 - val_auc: 0.6579 - lr: 0.0010
Epoch 7/100
3060/3060 [==============================] - 640s 209ms/step - loss: 0.2678 - auc: 0.664